In [2]:
:dep lambdaworks-stark = { path = "." }
:dep lambdaworks-math = { git = "https://github.com/lambdaclass/lambdaworks", rev = "a17b951" }
:dep lambdaworks-crypto = { git = "https://github.com/lambdaclass/lambdaworks", rev = "a17b951" }
:dep num-integer

In [3]:
use std::ops::Range;

use lambdaworks_math::field::fields::{
    fft_friendly::stark_252_prime_field::Stark252PrimeField as F,
    u64_prime_field::{F17, FE17},
};
use lambdaworks_stark::{
    cairo::{
        air::{
            generate_cairo_proof, verify_cairo_proof, MemorySegment, MemorySegmentMap,
            PublicInputs, FRAME_DST_ADDR, FRAME_OP0_ADDR, FRAME_OP1_ADDR, FRAME_PC,
        },
        cairo_layout::CairoLayout,
        execution_trace::build_main_trace,
        runner::run::{
            cairo0_program_path, cairo1_program_path, generate_prover_args, run_program,
            CairoVersion,
        },
    },
    starks::{
        example::{
            dummy_air::{self, DummyAIR},
            fibonacci_2_columns::{self, Fibonacci2ColsAIR},
            fibonacci_rap::{fibonacci_rap_trace, FibonacciRAP, FibonacciRAPPublicInputs},
            quadratic_air::{self, QuadraticAIR, QuadraticPublicInputs},
            simple_fibonacci::{self, FibonacciAIR, FibonacciPublicInputs},
        },
        proof::options::{ProofOptions, SecurityLevel},
        prover::prove,
        trace::TraceTable,
        verifier::verify,
    },
    FE,
};

use lambdaworks_crypto::fiat_shamir::transcript::Transcript;
use lambdaworks_math::field::{element::FieldElement, traits::IsFFTField};

use lambdaworks_stark::starks::{
    constraints::boundary::{BoundaryConstraint, BoundaryConstraints},
    context::AirContext,
    frame::Frame,
    proof::options::ProofOptions,
    trace::TraceTable,
    traits::AIR,
};
use std::sync::atomic::{AtomicUsize, AtomicU64, AtomicIsize, Ordering::SeqCst};
use std::time::{Instant, Duration};
use num_integer::binomial;

In [14]:
use std::sync::atomic::{AtomicUsize, AtomicU64, AtomicIsize, Ordering::SeqCst};

In [78]:
// Number of columns to use in the AIR.
static N_COLS: AtomicUsize = AtomicUsize::new(0);

// Factors for the transitions. 
static FIRST_FACTOR: AtomicU64 = AtomicU64::new(2);
static SECOND_FACTOR: AtomicIsize = AtomicIsize::new(1);

pub fn get_factors() -> (u64, isize) {
    (FIRST_FACTOR.load(SeqCst), SECOND_FACTOR.load(SeqCst))
}

// Jump factor para las subsecuencias. Por ejemplo, en vez de tener
// la subsecuencia 1,2,3,4,5; con un jumpFactor de 2, se tendrian solo los numeros 1,3,5,...
pub fn update_factors(jump_factor: u32) -> (FieldElement<F>, FieldElement<F>) {
    let mut first_factor = 0;
    for k in 0..(jump_factor + 1) {
        if k % 2 == 0 {
            first_factor += binomial(jump_factor, k) * u32::pow(2, k / 2);
        }
    }
    first_factor *= 2;
    let second_factor = if jump_factor % 2 == 1 {1} else {-1};
    FIRST_FACTOR.store(first_factor as u64, SeqCst);
    SECOND_FACTOR.store(second_factor, SeqCst);
    let c0 = FieldElement::<F>::from(first_factor as u64);
    let c1 = if second_factor == -1 {
        FieldElement::<F>::zero() - FieldElement::<F>::one()
    } else {// Vale 1.
        FieldElement::<F>::one()
    };
    (c0, c1)
}


pub fn change_n_cols(new_value: usize) -> usize {
    N_COLS.store(new_value, SeqCst);
    new_value
}

pub fn get_n_cols() -> usize {
    N_COLS.load(SeqCst)
}


#[derive(Clone)]
pub struct PellAIR<F>
where
    F: IsFFTField,
{
    context: AirContext,
    trace_length: usize,
    pub_inputs: PellPublicInputs<F>,
    n_columns: usize,
    first_factor: FieldElement<F>,
    second_factor: FieldElement<F>,
}

#[derive(Clone, Debug)]
pub struct PellPublicInputs<F>
where
    F: IsFFTField,
{
    pub a0: FieldElement<F>,
    pub a1: FieldElement<F>,
}

impl<F> AIR for PellAIR<F>
where
    F: IsFFTField,
{
    type Field = F;
    type RAPChallenges = ();
    type PublicInputs = PellPublicInputs<Self::Field>;

    fn new(
        trace_length: usize,
        pub_inputs: &Self::PublicInputs,
        proof_options: &ProofOptions,
    ) -> Self {
        let n_columns = get_n_cols();
        let transition_offsets = if n_columns > 1 {vec![0, 1]} else {vec![0, 1, 2]};
        let transition_exemptions = if n_columns == 1 {
            vec![2]
        } else {
            vec![1, 1].into_iter().chain(std::iter::repeat(0).take(n_columns - 2)).collect()
        };
        let context = AirContext {
            proof_options: proof_options.clone(),
            trace_columns: n_columns,
            transition_degrees: vec![1; n_columns], // usamos la misma transicion en cada columna. 
            transition_offsets: transition_offsets,
            num_transition_constraints: n_columns,
            transition_exemptions: transition_exemptions,
            num_transition_exemptions: 1,
        };
        let (c0, c1) = get_factors();
        let second_factor = if c1 == -1 {
            FieldElement::<F>::zero() - FieldElement::<F>::one()
        } else {// Vale 1.
            FieldElement::<F>::one()
        };
        Self {
            pub_inputs: pub_inputs.clone(),
            context,
            trace_length,
            n_columns,
            first_factor: FieldElement::<F>::from(c0),
            second_factor,
        }
    }

    fn composition_poly_degree_bound(&self) -> usize {
        self.trace_length()
    }

    fn build_auxiliary_trace(
        &self,
        _main_trace: &TraceTable<Self::Field>,
        _rap_challenges: &Self::RAPChallenges,
    ) -> TraceTable<Self::Field> {
        TraceTable::empty()
    }

    fn build_rap_challenges<T: Transcript>(&self, _transcript: &mut T) -> Self::RAPChallenges {}

    fn compute_transition(
        &self,
        frame: &Frame<Self::Field>,
        _rap_challenges: &Self::RAPChallenges,
    ) -> Vec<FieldElement<Self::Field>> {
        let first_row = frame.get_row(0);
        let second_row = frame.get_row(1);
        if self.n_columns > 1 {
            let mut transitions = vec![];
            // La primera restriccion es tal que s_{0, i+1} = 2 * s_{ncols - 1, i} + s_{ncols - 2, i}.
            transitions.push(second_row[0].clone() - self.first_factor.clone() * first_row[self.n_columns - 1].clone() - self.second_factor.clone() * first_row[(2 * self.n_columns - 2) % self.n_columns].clone());
            // La segunda restriccion tambien usa la fila anterior siempre. 
            transitions.push(second_row[1].clone() - self.first_factor.clone() * second_row[0].clone() - self.second_factor.clone() * first_row[self.n_columns - 1].clone());
            // Las demas transiciones son sobre la misma fila pero de las anteriores dos columnas.
            for i in 2..self.n_columns {
                transitions.push(second_row[i].clone() - self.first_factor.clone() * second_row[i-1].clone() - self.second_factor.clone() * second_row[i-2].clone());
            }
            return transitions;
        }
        
        let third_row = frame.get_row(2);

        vec![third_row[0].clone() - self.first_factor.clone() * second_row[0].clone() - self.second_factor.clone() * first_row[0].clone()]
    }

    fn boundary_constraints(
        &self,
        _rap_challenges: &Self::RAPChallenges,
    ) -> BoundaryConstraints<Self::Field> {
        let a0 = BoundaryConstraint::new(0, 0, self.pub_inputs.a0.clone());
        // si se hace con mas de una columna, la segunda restriccion es el primer
        // valor de la segunda columna.
        let a1 = if self.n_columns > 1 { 
            BoundaryConstraint::new(1, 0, self.pub_inputs.a1.clone())
        } else { 
            BoundaryConstraint::new(0, 1, self.pub_inputs.a1.clone())
        };

        BoundaryConstraints::from_constraints(vec![a0, a1])
    }

    fn number_auxiliary_rap_columns(&self) -> usize {
        0
    }

    fn context(&self) -> &AirContext {
        &self.context
    }

    fn trace_length(&self) -> usize {
        self.trace_length
    }

    fn pub_inputs(&self) -> &Self::PublicInputs {
        &self.pub_inputs
    }
}



In [79]:
pub fn pell_trace<F: IsFFTField>(
    initial_values: [FieldElement<F>; 2],
    trace_length: usize,
    n_columns: usize,
    first_factor: FieldElement<F>,
    second_factor: FieldElement<F>,
) -> TraceTable<F> {
    let mut ret: Vec<Vec<FieldElement<F>>> = vec![vec![]; n_columns];
    ret[0].push(initial_values[0].clone());
    
    // si hay mas de una columna, es el primer elemento de la segunda columna.
    // Sino, en la unica columna que hay 
    ret[1 % n_columns].push(initial_values[1].clone());
    let mut a_n_1 = initial_values[1].clone();
    let mut a_n_2 = initial_values[0].clone();

    for i in 2..(trace_length * n_columns) {
        let res = first_factor.clone() * a_n_1.clone() + second_factor.clone() * a_n_2.clone();
        ret[i % n_columns].push(res.clone());
        a_n_2 = a_n_1;
        a_n_1 = res;
    }
    TraceTable::new_from_cols(&ret)
}

In [84]:
fn run_test_prove(
    n_cols: usize,
    trace_length: usize,
    first_factor: FieldElement<F>,
    second_factor: FieldElement<F>,
    expected_result: FieldElement<F>,
    pub_inputs: PellPublicInputs<F>,
) -> (Duration, Duration, Duration) {
    let mut now = Instant::now();
    let trace = pell_trace([pub_inputs.a0.clone(), pub_inputs.a1.clone()], trace_length, n_cols, first_factor, second_factor);
    //assert_eq!(expected_result, trace.get(trace_length - 1, n_cols - 1));
    let trace_time = now.elapsed();
    
    let proof_options = ProofOptions::default_test_options();
    
    now = Instant::now();
    let proof = prove::<F, PellAIR<F>>(&trace, &pub_inputs, &proof_options).unwrap();
    let proof_time = now.elapsed();
    
    now = Instant::now();
    assert!(verify::<F, PellAIR<F>>(
        &proof,
        &pub_inputs,
        &proof_options
    ));
    let verify_time = now.elapsed();
    
    (trace_time, proof_time, verify_time)
}

In [85]:
let n_cols = 1;
let trace_length = 1024;
let power_2_tracelen = 10;
let jump_factor = 1;
// Validacion extra que agregamos: 
// todas las opciones deberian calcular lo mismo, y llegar al mismo resultado.
let (original_c0, original_c1) = update_factors(1);
let original_pub_inputs = PellPublicInputs {
    a0: FE::zero(),
    a1: FE::one(),
};
let trace = pell_trace([original_pub_inputs.a0.clone(), 
                        original_pub_inputs.a1.clone()],
                        trace_length, n_cols, original_c0.clone(), original_c1.clone());
let expected_result = trace.get(trace_length - 1, n_cols - 1);

let n_runs = 10;


for jump_factor in 0..power_2_tracelen {
    let jump_len = u32::pow(2,jump_factor);
    // cambia la ecuacion de recurrencia al ser una subsecuencia.
    let (c0, c1) = update_factors(jump_len);
    // calculamos el segundo input que tambien cambia.
    let n_cols = 1;
    let trace = pell_trace([original_pub_inputs.a0.clone(), 
                            original_pub_inputs.a1.clone()],
                            jump_len as usize+1, n_cols, original_c0.clone(), original_c1.clone());
    println!("{}", trace.get(jump_len as usize, n_cols - 1));
    let a1 = trace.get(jump_len as usize, n_cols - 1);
    let pub_inputs = PellPublicInputs {
        a0: original_pub_inputs.a0.clone(),
        a1: a1,
    };
    
    for col_factor in 0..power_2_tracelen-jump_factor+1 {
        let n_cols = change_n_cols(usize::pow(2,col_factor));
        let trace_length = usize::pow(2,power_2_tracelen - col_factor - jump_factor);
        let (mut trace_time, mut proof_time, mut verify_time) = (Duration::ZERO, Duration::ZERO, Duration::ZERO);  
        println!("#JUMPS: {} - # COLS: {} - #TRACELEN: {} - INPUT: {}", jump_len, n_cols, trace_length, a1.clone());
        for i in 0..n_runs {
            let (t, p, v) = run_test_prove(n_cols, trace_length, c0.clone(), c1.clone(), expected_result.clone(), pub_inputs.clone());
            trace_time += t; proof_time += p; verify_time += v;
        }
        println!("#JUMPS: {} - # COLS: {} #TRACELEN: {} - Tiempos: {}, {}, {}", jump_len, n_cols, trace_length, trace_time.as_micros(), proof_time.as_millis(), verify_time.as_millis());

    }
}

0x1
#JUMPS: 1 - # COLS: 1 - #TRACELEN: 1024 - INPUT: 0x1
#JUMPS: 1 - # COLS: 1 #TRACELEN: 1024 - Tiempos: 652, 361, 24
#JUMPS: 1 - # COLS: 2 - #TRACELEN: 512 - INPUT: 0x1
#JUMPS: 1 - # COLS: 2 #TRACELEN: 512 - Tiempos: 716, 189, 14
#JUMPS: 1 - # COLS: 4 - #TRACELEN: 256 - INPUT: 0x1
#JUMPS: 1 - # COLS: 4 #TRACELEN: 256 - Tiempos: 700, 108, 8
#JUMPS: 1 - # COLS: 8 - #TRACELEN: 128 - INPUT: 0x1
#JUMPS: 1 - # COLS: 8 #TRACELEN: 128 - Tiempos: 706, 74, 5
#JUMPS: 1 - # COLS: 16 - #TRACELEN: 64 - INPUT: 0x1
#JUMPS: 1 - # COLS: 16 #TRACELEN: 64 - Tiempos: 795, 54, 4
#JUMPS: 1 - # COLS: 32 - #TRACELEN: 32 - INPUT: 0x1
#JUMPS: 1 - # COLS: 32 #TRACELEN: 32 - Tiempos: 802, 43, 4
#JUMPS: 1 - # COLS: 64 - #TRACELEN: 16 - INPUT: 0x1
#JUMPS: 1 - # COLS: 64 #TRACELEN: 16 - Tiempos: 939, 41, 4
#JUMPS: 1 - # COLS: 128 - #TRACELEN: 8 - INPUT: 0x1
#JUMPS: 1 - # COLS: 128 #TRACELEN: 8 - Tiempos: 961, 45, 7
#JUMPS: 1 - # COLS: 256 - #TRACELEN: 4 - INPUT: 0x1
#JUMPS: 1 - # COLS: 256 #TRACELEN: 4 - Tiempos: 8

thread '<unnamed>' panicked at 'attempt to multiply with overflow', src/lib.rs:234:29
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8ede3aae28fe6e4d52b38157d7bfe0d3bceef225/library/std/src/panicking.rs:593:5
   1: core::panicking::panic_fmt
             at /rustc/8ede3aae28fe6e4d52b38157d7bfe0d3bceef225/library/core/src/panicking.rs:67:14
   2: core::panicking::panic
             at /rustc/8ede3aae28fe6e4d52b38157d7bfe0d3bceef225/library/core/src/panicking.rs:117:5
   3: <unknown>
   4: <unknown>
   5: <unknown>
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


In [75]:
get_factors()

(2, 1)

In [76]:
update_factors(1);
get_factors()

(2, 1)

In [82]:
update_factors(2);
get_factors()

(6, -1)